# install librarys

In [1]:
!pip install -U langchain_openai
!pip install -U huggingface_hub
!pip install -U langchain_community
!pip install -U pydantic

## connect to OpenAI gpt API
To connect to the OpenAI GPT API, we utilized [AvalAi](https://avalai.ir/). After signing in, we generated an API key specifically for our project.

In [2]:
from langchain_openai import ChatOpenAI,OpenAI
from langchain_community.callbacks import get_openai_callback

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello world!"},
]
model_name = "gpt-4o-mini" # in this case we want to use gpt-4o-mini


llm = ChatOpenAI(
    model=model_name,
    base_url="https://api.avalai.ir/v1",
    temperature=None,
    max_tokens=2000, #token limiter
    timeout=None,
    max_retries=0,



    api_key="aa-qQ83RCj6StMiojqP9GQXDzWUNENhEbKfBf0n2dKusfAKp7I7"
)
# this is testing the API connection and tracking token usage
with get_openai_callback() as cb:
  response = llm.invoke(messages)
  print(cb)



Tokens Used: 30
	Prompt Tokens: 20
		Prompt Tokens Cached: 0
	Completion Tokens: 10
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $8.999999999999999e-06


## Creating a pydantic class for better output format

In [3]:
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Dict


class ProductReviewGap(BaseModel):
    """Represents gaps and review mentions for a single product."""
    product_name: str = Field(description="Name of the product")
    review_mentions: List[str] = Field(..., description="Features or topics mentioned by customers in reviews")
    missing_in_description: List[str] = Field(..., description="Features mentioned in reviews but missing from product description (content gaps)")
    # product name must not be empty
    @field_validator("product_name")
    def validate_product_name(cls, v):
        """Ensure product name is not empty or only whitespace."""
        if not v.strip():
            raise ValueError("product_name cannot be empty or whitespace")
        return v


class ContentGapAnalysisResult(BaseModel):
    """Main model for the content gap analysis result."""
    common_features: List[str] = Field(..., description="Features common to all products")
    unique_features: Dict[str, List[str]] = Field(..., description="Unique features for each product; dictionary key = product name")
    customer_gaps: List[ProductReviewGap] = Field(..., description="List of gaps and review mentions for each product")
    marketing_insight: str = Field(description="A simple, business-oriented summary for the marketing team")
    #key feild must not be empty
    @field_validator("marketing_insight")
    def validate_not_empty(cls, v, info):
        """Ensure marketing insight is not empty."""
        if not v or not v.strip():
            raise ValueError(f"{info.field_name} cannot be empty")
        return v
    #Checking the compatibility of product names between different parts
    @model_validator(mode="after")
    def validate_product_consistency(self):
        """Ensure all products in customer_gaps exist in unique_features."""
        if self.unique_features and self.customer_gaps:
            product_names_from_unique = set(self.unique_features.keys())
            product_names_from_gaps = {gap.product_name for gap in self.customer_gaps}
            missing = product_names_from_gaps - product_names_from_unique
            if missing:
                raise ValueError(
                    f"Products in customer_gaps not found in unique_features: {', '.join(missing)}"
                )
        return self


## Translating the key words for Farsi output

In [4]:
KEY_MAP = {
    "نام_محصول": "product_name",
    "اشارات_در_نقد": "review_mentions",
    "مفقود_در_توضیحات": "missing_in_description",
    "ویژگی‌های_مشترک": "common_features",
    "ویژگی‌های_منحصر_به_فرد": "unique_features",
    "شکاف‌های_مشتری": "customer_gaps",
    "بینش_بازاریابی": "marketing_insight",
    "between_marketing_insights": "marketing_insight"
}

def translate_keys(data):
    if isinstance(data, dict):
        return {KEY_MAP.get(k, k): translate_keys(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_keys(item) for item in data]
    return data

In [5]:
import re

def force_json_closure(text: str) -> str:
    match = re.search(r"\{.*\}", text, re.S)
    if match:
        return match.group()
    return "{}"
# better output format with EN-FA sentences
def normalize_mixed_text(text: str) -> str:
    text = re.sub(r'[\u200c\s]+', ' ', text)
    text = re.sub(r'\s+([,.!?;:])', r'\1', text)
    text = re.sub(r'([،؛؟])\s*', r'\1 ', text)
    text = re.sub(r'([آ-ی])([A-Za-z0-9])', r'\1 \2', text)
    text = re.sub(r'([A-Za-z0-9])([آ-ی])', r'\1 \2', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    return text.strip()

# Prompting The Model

In [6]:
import json
from pydantic import ValidationError

def analyze_content_gaps(input_file: str, output_file: str ):
    with open(input_file, "r", encoding="utf-8") as f:
        content = f.read()

    messages = [
    {
        "role": "system",
        "content": """
شما یک دستیار هوش مصنوعی برای **تحلیل شکاف محتوا** هستید.

وظیفه شما مقایسه توضیحات محصول و نظرات مشتریان است.

سپس شکاف‌ها و بینش‌ها را در قالبی ساختاریافته شناسایی کنید.

### فرمت خروجی (JSON دقیقاً از این الگو پیروی می‌کند)

{
"ویژگی‌های_مشترک": [
"string",
"... ویژگی‌های بیشتر"
],
"ویژگی‌های_منحصر_به_فرد": {
"نام_محصول1": ["ویژگی1", "ویژگی2"],
"نام_محصول2": ["ویژگیA"]
},
"شکاف‌های_مشتری": [
{
"نام_محصول": "string",
"اشارات_در_نقد": [
"ویژگی1",
"ویژگی2"
],
"مفقود_در_توضیحات": [
"ویژگیX",
"ویژگیY"
]
}
],
"بینش_بازاریابی": "خلاصه‌ای کوتاه به زبان تجاری،
برجسته کردن آنچه که بازاریابی باید بر آن تأکید کند یا بهبود بخشد."

}

### قوانین سختگیرانه
۱. "ویژگی‌های_مشترک": باید فقط شامل ویژگی‌هایی باشد که در توضیحات **همه محصولات** آمده‌اند.

- اگر یک ویژگی حتی در یک محصول وجود ندارد، آن را ذکر نکنید.

۲. "ویژگی‌های_منحصر_به_فرد": باید فقط شامل ویژگی‌هایی باشد که **منحصراً مختص آن محصول** هستند.

- اگر یک ویژگی بین دو یا چند محصول مشترک است، آن را اینجا ذکر نکنید.

۳. "شکاف‌های_مشتری":

- برای هر محصول، موضوعاتی را که مشتریان در نقدها ذکر کرده‌اند، فهرست کنید.

- در قسمت "مفقود_در_توضیحات"، فقط موضوعاتی را در نقدها ذکر کنید که در توضیحات آن محصول ذکر نشده‌اند.

۴. "بینش_بازاریابی": ۳-۴ جمله به زبان انگلیسی ساده تجاری بنویسید و خلاصه‌ای از مهمترین یافته‌ها و آنچه بازاریابی باید روی آن تمرکز کند، ارائه دهید.

### قوانین خروجی
- خروجی باید یک شیء **معتبر JSON** باشد که بتواند مستقیماً توسط json.loads() تجزیه شود.

- هرگز آرایه‌ها یا اشیاء را بسته نشده رها نکنید.

- پاسخ را قطع نکنید. همیشه شیء JSON کامل را برگردانید.

- برای کلیدها و رشته‌ها از علامت نقل قول دوتایی استفاده کنید.

- بدون توضیح، بدون نظر، بدون متن اضافی خارج از JSON.

- همیشه آرایه‌ها [] و اشیاء {} را به درستی ببندید.

- تعداد محصولات ورودی همیشه ثابت نیست.

- اگر هیچ نظری برای یک محصول وجود ندارد، customer_gaps را خالی بگذارید.
"""
    },
          #Example for better performance

         {"role": "user", "content": """محصولات:
هارد اکسترنال وسترن دیجیتال مای پاسپورت، ظرفیت ۱ ترابایت (digikala):

توضیحات:
هارد اکسترنال مای پاسپورت محصولی از شرکت وسترن دیجیتال است که از طریق رابط USB3.0 به رایانه شما متصل می‌شود و به شما کمک می‌کند تا اطلاعات را با سرعت بالا ذخیره کنید. ظرفیت این هارد یک ترابایت است و برای کاربرانی که اطلاعات زیادی برای ذخیره دارند، گزینه بسیار مناسبی است. هارد اکسترنال وسترن دیجیتال مای پاسپورت با تمام سیستم عامل‌های رایج از جمله نسخه‌های مختلف سیستم عامل ویندوز سازگار است. هارد اکسترنال مای پاسپورت در رنگ‌های مختلف عرضه می‌شود، بنابراین برای هر سلیقه‌ای انتخاب مناسبی وجود خواهد داشت. نسل چهارم هارد اکسترنال‌های مای پاسپورت از سال ۲۰۱۹ با طراحی زیبا وارد بازار شده است. سرعت انتقال اطلاعات با هارد اکسترنال وسترن دیجیتال ۵ گیگابایت بر ثانیه است. شایان ذکر است که این هارد اکسترنال وسترن دیجیتال به دلیل وزن سبک خود به راحتی قابل حمل است. سرعت چرخش این هارد اکسترنال نیز ۵۴۰۰ دور در دقیقه است.


نقد و بررسی‌ها:
- "دوستان، آیا این هارد دیسک می‌تواند به تلفن همراه متصل شود؟ اندروید؟"
- "سلام، آیا از Xbox Series S پشتیبانی می‌کند؟"
- "سلام، گارانتی از چه شرکتی است؟ و آیا معتبر است؟"

هارد اکسترنال Western Digital My Passport، ظرفیت ۱ ترابایت (technolife):

توضیحات:
وزن: ۱۲۲.۴ گرم
ابعاد: ۱۰۷.۲x۷۵x۱۱.۲ میلی‌متر
نوع رابط: USB 3.0
ظرفیت: ۱ ترابایت
اندازه هد: ۲.۵ اینچ
سرعت انتقال داده: ۵ گیگابیت بر ثانیه
نشانگر LED: ندارد
سایر مشخصات: پشتیبان‌گیری خودکار از طریق نرم‌افزار، محافظت با رمز عبور برای امنیت، پشتیبانی از رمزگذاری AES 256 بیتی

نقد و بررسی‌ها:
- "ببخشید، آیا می‌توان از آن با ویندوز ۱۱ استفاده کرد؟!"
- "سلام، می‌خواستم بدانم که آیا می‌توان آن را به مک‌بوک متصل کرد؟"
- "سلام، صبح بخیر. من یک فیلمبردار هستم. به نظر شما یک هارد اکسترنال ۱ ترابایتی برای ویدیوهای اینستاگرام کافی است؟"

هارد اکسترنال وسترن دیجیتال مای پاسپورت، ظرفیت ۱ ترابایت (arbabashop):

توضیحات:
بررسی هارد اکسترنال WD My Passport 1TB
هارد اکسترنال وسترن دیجیتال مای پاسپورت ۱ ترابایتی یکی از محبوب‌ترین و پرفروش‌ترین هارد دیسک‌های قابل حمل موجود در بازار است. این محصول با طراحی کوچک و سبک خود، گزینه‌ای ایده‌آل برای افرادی است که به دنبال فضایی امن برای ذخیره اطلاعات شخصی، کاری یا پشتیبان‌گیری هستند. برند WD سال‌هاست که به عنوان یکی از معتبرترین تولیدکنندگان تجهیزات ذخیره‌سازی شناخته می‌شود و سری My Passport یکی از پرچمداران آن است.

طراحی و ساخت
بدنه جمع و جور و سبک، مناسب برای حمل روزانه
موجود در رنگ‌های متنوع برای سلیقه‌های مختلف
استفاده از پورت Micro-B (سازگار با USB 3.2 Gen 1 و USB 2.0)
کیفیت ساخت خوب با طراحی ضد لغزش برای جلوگیری از لغزش روی سطوح
مشخصات فنی
ظرفیت ذخیره‌سازی: ۱ ترابایت
رابط اتصال: USB 3.2 Gen 1 (سرعت تا ۵ گیگابیت بر ثانیه)
امنیت: رمزگذاری سخت‌افزاری AES 256 بیتی + قابلیت رمزگذاری با رمز عبور
نرم‌افزار: WD Backup، WD Discovery، WD Security برای مدیریت و پشتیبان‌گیری
ابعاد: ۱۰۷ × ۷۵ × ۱۱ میلی‌متر
وزن: حدود ۱۲۰ گرم
گارانتی: ۳ سال
عملکرد و سرعت
طبق آزمایش‌ها، سرعت انتقال داده این هارد دیسک:
خواندن: حدود ۱۲۰ تا ۱۳۵ مگابایت بر ثانیه
نوشتن: حدود ۱۱۰ تا ۱۲۵ مگابایت بر ثانیه
این سرعت برای ... بسیار خوب است هارد دیسک است و به راحتی می‌تواند فیلم، عکس، موسیقی و فایل‌های بزرگ را ذخیره کند.

مزایا
✅ قیمت مقرون به صرفه و اقتصادی در مقایسه با ظرفیت
✅ طراحی سبک و قابل حمل
✅ پشتیبانی از رمزگذاری سخت‌افزاری برای امنیت داده‌ها
✅ نرم‌افزار مدیریت و پشتیبان‌گیری WD
✅ سازگاری با ویندوز و مک
"""},
         {"role": "assistant", "content":"""{
  "common_features": [
    "ظرفیت 1 ترابایت",
    "طراحی سبک و قابل حمل",
    "رابط USB 3.0",
    "سرعت انتقال داده تا 5 گیگابیت بر ثانیه",
    "موجود در رنگ‌های مختلف"
  ],
  "unique_features": {
    "هارد اکسترنال وسترن دیجیتال مای پاسپورت (دیجیکالا)": [
      "سرعت چرخش 5400 دور در دقیقه",
      "سازگار با تمام سیستم عامل‌های رایج",
      "معرفی شده در سال 2019 (نسل چهارم)"
    ],
    "هارد اکسترنال Western Digital My Passport (تکنولایف)": [
      "بدون نشانگر LED",
      "پشتیبان‌گیری خودکار از طریق نرم‌افزار",
      "محافظت با رمز عبور",
      "رمزگذاری AES 256 بیتی"
    ],
    "هارد اکسترنال وسترن دیجیتال مای پاسپورت (اربب شاپ)": [
      "طراحی ضد لغزش",
      "پورت USB 3.2 Gen 1 + Micro-B",
      "نرم‌افزارهای WD Backup, WD Discovery, WD Security",
      "3 سال گارانتی",
      "سرعت خواندن 120-135 مگابایت بر ثانیه، سرعت نوشتن 110-125 مگابایت بر ثانیه",
      "قیمت مقرون به صرفه"
    ]
  },
  "customer_gaps": [
    {
      "نام_محصول": "هارد اکسترنال وسترن دیجیتال مای پاسپورت (دیجیکالا)",
      "اشارات_در_نقد": [
        "سازگاری با تلفن همراه (اندروید)",
        "پشتیبانی از Xbox Series S",
        "شرکت گارانتی کننده"
      ],
      "مفقود_در_توضیحات": [
        "سازگاری با تلفن همراه",
        "سازگاری با Xbox Series S",
        "جزئیات گارانتی"
      ]
    },
    {
      "نام_محصول": "هارد اکسترنال Western Digital My Passport (تکنولایف)",
      "اشارات_در_نقد": [
        "سازگاری با ویندوز 11",
        "سازگاری با مک‌بوک",
        "کفایت حافظه برای فیلمبرداران"
      ],
      "مفقود_در_توضیحات": [
        "سازگاری با ویندوز 11",
        "سازگاری با مک‌بوک",
        "راهنمایی برای استفاده در فیلمبرداری"
      ]
    },
    {
      "نام_محصول": "هارد اکسترنال وسترن دیجیتال مای پاسپورت (اربب شاپ)",
      "اشارات_در_نقد": [],
      "مفقود_در_توضیحات": []
    }
  ],
  "بینش_بازاریابی": "همه صفحات محصول اصلی ویژگی‌هایی مانند ظرفیت 1 ترابایت، قابلیت حمل و عملکرد USB 3.0 را برجسته می‌کنند، اما هر کدام نقاط فروش منحصر به فردی مانند امنیت، طراحی بدنه یا گارانتی دارند. نظرات مشتریان به طور مداوم در مورد سازگاری با دستگاه‌های تلفن همراه، کنسول‌های بازی، ویندوز 11 و مک‌بوک سوال می‌پرسند - با این حال هیچ یک از توضیحات به صراحت به این نگرانی‌ها پاسخ نمی‌دهند. بازاریابی باید بر سازگاری بین پلتفرم‌ها تأکید کند، جزئیات واضح گارانتی را درج کند و راهنمایی برای موارد استفاده خاص مانند بازی و تولید محتوا ارائه دهد. برجسته کردن ویژگی‌های امنیتی قوی و پشتیبانی نرم‌افزاری نیز محصول را در بازار رقابتی متمایز خواهد کرد."}""" },

         #using own data
         {"role": "user","content": content}
    ]

    try:
        with get_openai_callback() as cb:
            response = llm.invoke(messages)

        print(cb)  # usage stats

        cleaned = force_json_closure(response.content.strip())
        data = json.loads(cleaned)
        data = translate_keys(data) # Keep the fix_keys function call
        result = ContentGapAnalysisResult(**data)
        with open(output_file, 'w',encoding="utf-8") as f:
          json.dump(result.model_dump(by_alias=True), f, indent=2, ensure_ascii=False)
        print("---------")
        print(f"✅ نتیجه با موفقیت در فایل ذخیره شد: {output_file}")

    except ValidationError as e:
        print("\n خروجی مدل با ساختار مورد انتظار هم‌خوانی ندارد!")
        print("-" * 40)
        print("جزئیات خطا:")
        print(e.errors())
        print("-" * 40)
        print(" خروجی خام مدل:\n", response.content)

    except Exception as e:
        print("\n⚠️ خطای غیرمنتظره‌ای رخ داد!")
        print("-" * 40)
        print("🔍 جزئیات خطا:", repr(e))
        print("-" * 40)
        print(" خروجی خام مدل:\n", response.content)

In [ ]:
result = analyze_content_gaps("input.txt","output.json")

Tokens Used: 5671
	Prompt Tokens: 4931
		Prompt Tokens Cached: 0
	Completion Tokens: 740
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0011836499999999998
---------
✅ نتیجه با موفقیت در فایل ذخیره شد: output.txt
